In [1]:
%%time
import numpy as np
import sympy as sp
import cvxpy as cp
from scipy.linalg import null_space
from scipy.linalg import sqrtm
from numpy.linalg import inv
from numpy.linalg import det
import itertools
from dataclasses import dataclass
##########################
threshold = 1e-10
###################################
@dataclass
class Poly_Face:
    # Assign to each face the equations defining it 
    equs: list[int]
    # Assign to each face its codimension
    codim: int
    # Assign to each face a list of proper subfaces
    subfaces: list[int]
    # Assign to each face a sample point in its interior
    sample_point: np.ndarray

@dataclass
class Find_Intersection:
    # The sample point. Zero matrix if not intersect
    sample_point: np.ndarray
    # If they intersect
    is_intersection: bool

@dataclass
class Ridge_Cycles:
    # a list of indices of ridges in List_faces
    ridge: list[int]
    # a list of indices of facet pairings in bises_active
    pairing: list[int]

@dataclass
class Word_Bis:
    # Matrix for the "word"
    word: np.ndarray
    # Matrix for the "bisector"
    bis: np.ndarray
##########################
# First algorithm: check if a matrix is positive
# Return True if it is
def is_positive_definite(matrix, threshold=1e-10):
    if not isinstance(matrix, np.ndarray):
        raise ValueError("Input must be a numpy array.")
    if matrix.shape[0] != matrix.shape[1]:
        return False  # The matrix is not square

    try:
        # Perform Cholesky decomposition
        min_diag = np.min(np.linalg.eigvalsh(matrix))

        # If the minimum diagonal entry is too small, consider the matrix nearly singular
        return min_diag > threshold
    except np.linalg.LinAlgError:
        return False  # Cholesky failed, not positive definite
##########################
# Second algorithm: the positive perpendicular vector
def compute_vector(d):
# Step 1: Check if there's at least one positive and one negative element
    positive_indices = [i for i, x in enumerate(d) if x > threshold]
    negative_indices = [i for i, x in enumerate(d) if x < -threshold]

    if not positive_indices or not negative_indices:
        # If no positive or no negative elements, return None
        return None, None

    # Step 2: Find the positive and negative components and their indices
    positive_index = positive_indices[0]  # Take the first positive
    negative_index = negative_indices[0]  # Take the first negative

    # Step 3: Rearrange the components of d such that positive is d0, and negative is d1
    d_sorted = [0, 0, 0]
    d_sorted[0] = d[positive_index]  # positive becomes d0
    d_sorted[1] = d[negative_index]  # negative becomes d1
    # Find the remaining component and place it as d2
    d_sorted[2] = next(x for i, x in enumerate(d) if i != positive_index and i != negative_index)
    d0, d1, d2 = d_sorted  # d0 is positive, d1 is negative, d2 is the remaining

    # Step 4: Compute v based on the rearranged components
    if d2<0:
        v = [-d1/(np.sqrt(d0**2 + d1**2)) - d2/(np.sqrt(d0**2 + d2**2)), d0/(np.sqrt(d0**2 + d1**2)), d0/(np.sqrt(d0**2 + d2**2))]
        w = [-d1/(np.sqrt(d0**2 + d1**2)) + d2/(np.sqrt(d0**2 + d2**2)), d0/(np.sqrt(d0**2 + d1**2)), -d0/(np.sqrt(d0**2 + d2**2))]
    else:
        v = [-d1/(np.sqrt(d0**2 + d1**2)), d0/(np.sqrt(d0**2 + d1**2)) + d2/(np.sqrt(d1**2 + d2**2)), -d1/(np.sqrt(d1**2 + d2**2))]
        w = [-d1/(np.sqrt(d0**2 + d1**2)), d0/(np.sqrt(d0**2 + d1**2)) - d2/(np.sqrt(d1**2 + d2**2)), d1/(np.sqrt(d1**2 + d2**2))]
    # Step 5: Permute the vector back to the original order of components in d
    v_original_order = [0, 0, 0]
    v_original_order[positive_index] = v[0]
    v_original_order[negative_index] = v[1]
    # Place the remaining component in its original position
    v_original_order[3 - positive_index - negative_index] = v[2]
    
    w_original_order = [0, 0, 0]
    w_original_order[positive_index] = w[0]
    w_original_order[negative_index] = w[1]
    # Place the remaining component in its original position
    w_original_order[3 - positive_index - negative_index] = w[2]
    
    return v_original_order, w_original_order
##########################
# Function to apply abs to coefficients but not the variables
def apply_abs_to_coeffs(expr):
    # Extract coefficients of x, y, z
    coeff_dict = expr.as_coefficients_dict()
    
    # Create the new expression by applying abs to the coefficients
    return sum(sp.Abs(coef) * var for var, coef in coeff_dict.items())
##########################
def word_to_vector(word):
    vector = [word[0][0],word[1][1],word[2][2],word[0][1],word[0][2],word[1][2]]
    return vector
def vector_to_word(vector):
    word = np.array([[vector[0], vector[3]/2, vector[4]/2],
                     [vector[3]/2, vector[1], vector[5]/2],
                     [vector[4]/2, vector[5]/2, vector[2]]])
    return word
##########################
def word_to_vector_new(word):
    vector = [word[0][0],word[1][1],word[2][2],np.sqrt(2) * word[0][1],np.sqrt(2) * word[0][2],np.sqrt(2) * word[1][2]]
    return vector
##########################
# Third algorithm: check linearly independence
# Here we assume "vectors" is a linearly independent set
def is_linearly_independent(vectors, new_vector, threshold=1e-10):
    if len(vectors) == 0:
        return np.linalg.norm(new_vector) > threshold  # The first vector is always independent
    # Stack the current independent vectors into a matrix
    matrix = np.array(vectors).T  # Transpose so that each column is a vector    
    # Compute the projection of the new vector onto the space spanned by the existing vectors
    projection = matrix @ np.linalg.pinv(matrix) @ new_vector
    # Compute the difference (residual) between the new vector and its projection
    residual = new_vector - projection
    # If the residual is smaller than the threshold, it's dependent
    return np.linalg.norm(residual) > threshold
##########################
def linearly_independent_subset(vectors):
    indep_vectors = []
    for vector in vectors:
        if is_linearly_independent(indep_vectors, vector):
            indep_vectors.append(vector)
    return indep_vectors
##########################
def orth_matrix(matrices):
    vectors = [word_to_vector(matrix) for matrix in matrices]
    vectors = linearly_independent_subset(vectors)
    orth_vectors = null_space(np.array(vectors)).T.tolist()
    orth_matrices = [np.array(vector_to_word(orth_vector)) for orth_vector in orth_vectors]
    return orth_matrices
######################
# Third algorithm: find the positive definite one by dichotomy method
# matrix is 3*3, linear, in term of the variables
def sp_to_cp(M_sym, varx):
    free_syms = varx
    n_vars = len(free_syms)
    rows, cols = M_sym.shape
    # Step 2: Initialize coefficient matrices
    coeff_matrices = [np.zeros((rows, cols), dtype=np.float64) for _ in range(n_vars + 1)]  # [M0, M1, ..., Mn]
    # Step 3: Decompose each entry
    for i in range(rows):
        for j in range(cols):
            expr = sp.expand(M_sym[i, j])
            if expr.is_Add:
                terms = expr.as_ordered_terms()
            else:
                terms = [expr]
            for term in terms:
                found = False
                for idx, sym in enumerate(free_syms):
                    if term.has(sym):
                        coeff = term.coeff(sym)
                        coeff_matrices[idx + 1][i, j] = float(coeff)
                        found = True
                        break
                if not found:
                    coeff_matrices[0][i, j] = float(term)  # constant term
    M_recovered = coeff_matrices  # [M0, M1, M2, M3]
    x_cvx = cp.Variable(n_vars)
    M_cvx = M_recovered[0] + sum(x_cvx[i]*M_recovered[i+1] for i in range(n_vars))
    return M_cvx
def find_pos_def(mat_expr, varx):
    t = cp.Variable()
    l = len(varx)
    x = cp.Variable(l)
    M = sp_to_cp(mat_expr, varx)
    constraints = [M - t*np.eye(3) >> 0,
               x >= -1, x <= 1]
    prob = cp.Problem(cp.Maximize(t), constraints)
    prob.solve(solver=cp.SCS,eps=threshold,max_iters=50000)
    #,verbose=True
    if prob.value > 100*threshold:
        return M.value
    else:
        return None
##########################
# Finally: check if a set of hyperplanes in X_3 intersect
def find_positive_definite_intersection(words):
    ################## Part 1: Consider a linearly independent sublist
    word_to_vector = [[word[0][0],word[1][1],word[2][2],word[0][1],word[0][2],word[1][2]] for word in words]
    independent_vectors = []
    for vec in word_to_vector:
        if is_linearly_independent(independent_vectors, vec):# Check if the current vector is linearly independent based on previous ones
            independent_vectors.append(vec)
    indep_matrix = [[[vec[0],vec[3],vec[4]],
                    [vec[3],vec[1],vec[5]],
                    [vec[4],vec[5],vec[2]]] for vec in independent_vectors]
    indep_matrix_np = [np.array(mat) for mat in indep_matrix]
    indep_matrix_sp = [sp.Matrix(mat) for mat in indep_matrix]
    ################### Part 2: The non-case
    A = indep_matrix_np[0]
    eigenvalues, eigenvectors = np.linalg.eigh(A)
    ev_orth, ev_orth_neg = compute_vector(eigenvalues)
    if ev_orth is None:
        sample_point = np.zeros((3, 3)) 
        is_intersection = False
    else:
        n = len(indep_matrix)
        Q = eigenvectors
        D_orth = np.diag(ev_orth)
    ################### Part 3: The case if n = 1
        if n == 1:
            sample_point = Q @ D_orth @ Q.T
            is_intersection = True
    ################### Part 4: The case if n > 1, we will begin canceling the variables
        else:
            linearized_matrix = [Q.T @ mat @ Q for mat in indep_matrix_np]
            linearized_matrix_sp = [sp.Matrix(mat) for mat in linearized_matrix]
            x_v = sp.symbols('x1:5')
            x_v_new = x_v #The necessary variables
            diag_max = [a + abs(b) for a, b in zip(ev_orth, ev_orth_neg)]
            D_orth_neg = [np.diag(ev_orth_neg),\
                          np.array([[0, np.sqrt(diag_max[0]*diag_max[1]), 0],
                                    [np.sqrt(diag_max[0]*diag_max[1]), 0, 0],
                                    [0, 0, 0]]),\
                          np.array([[0, 0, np.sqrt(diag_max[0]*diag_max[2])],
                                    [0, 0, 0],
                                    [np.sqrt(diag_max[0]*diag_max[2]), 0, 0]]),\
                          np.array([[0, 0, 0],
                                    [0, 0, np.sqrt(diag_max[1]*diag_max[2])],
                                    [0, np.sqrt(diag_max[1]*diag_max[2]), 0]])]
            D_orth_sp = sp.Matrix(D_orth)
            D_orth_neg_sp = [sp.Matrix(mat) for mat in D_orth_neg]
            matrix_comb = sum((var * mat for mat, var in zip(D_orth_neg_sp, x_v)), start=D_orth_sp)
            # Remove extra variables
            for i in range(1,n):
                trace_matrix_prod = (matrix_comb * linearized_matrix_sp[i]).trace().expand()
                trace_coeffs = {var: trace_matrix_prod.coeff(var) for var in x_v}
                #If a nonzero constant appears, return to false since it is surely empty.
                if all(abs(coeff) < threshold for coeff in trace_coeffs.values()):
                    sample_point = np.zeros((3, 3)) 
                    is_intersection = False
                    return Find_Intersection(np.array(sample_point), is_intersection) 
                max_var = max(trace_coeffs, key=lambda v: abs(trace_coeffs[v]))
                x_sol = sp.solve(trace_matrix_prod, max_var)[0]  # Solve f = 0 for max_var
                x_v_new = tuple(var for var in x_v_new if var != max_var) # Drop max_var from x_v_new
                matrix_comb = matrix_comb.subs(max_var,x_sol)
            # If all variables are removed
            if n == 5:
                D = np.array(matrix_comb).astype(np.float64)
                if is_positive_definite(D):
                    sample_point = Q @ D @ Q.T
                    is_intersection = True
                else:
                    sample_point = np.zeros((3, 3)) 
                    is_intersection = False
            # Set the equations
            else:
                poly_comb = matrix_comb.det()
                poly_comb_coeff = poly_comb.as_coefficients_dict()
                all_zero = all(abs(coef) < threshold for coef in poly_comb_coeff.values())
                if all_zero:
                    sample_point = np.zeros((3, 3)) 
                    is_intersection = False
                else:
                    D = find_pos_def(matrix_comb, x_v_new)
                    if D is None:
                        sample_point = np.zeros((3, 3)) 
                        is_intersection = False
                    else:
                        sample_point = Q @ D @ Q.T
                        is_intersection = True
    if is_intersection:
        if not is_positive_definite(sample_point):
            is_intersection = False
            sample_point = np.zeros((3, 3))
        else:
            sample_point = sample_point/((det(sample_point)) ** (1/3))
    return Find_Intersection(np.array(sample_point), is_intersection)
################################################################
def remove_duplicates(matrices):
    unique_matrices = []
    for matrix in matrices:
        if not any(np.array_equal(matrix, unique_matrix) for unique_matrix in unique_matrices):
            unique_matrices.append(matrix)
    return unique_matrices
# Function to remove elements appearing in preceding lists
def remove_preceding_elements(current_list, preceding_lists):
    preceding_set = {tuple(map(tuple, matrix)) for lst in preceding_lists for matrix in lst}
    return [matrix for matrix in current_list if tuple(map(tuple, matrix)) not in preceding_set]
#################################################################
# Compute all words from input
def generator_to_words(generators, length):
    A = [np.array(generator) for generator in generators]
    n = length
    k = len(A)
    A_inv = [inv(matrix) for matrix in A]
    A = A + A_inv
    # List of words ending with each generator
    word_A = [[matrix] for matrix in A]
    # List of "old" words
    old_A = [[] for matrix in A]
    # List of "all" words
    all_A = [[] for matrix in A]
    # Beginning of the loop
    for _ in range(n-1):
        new_A = [[] for matrix in A]
        for i in range(2*k):
            for j in range(2*k):
                if (j-i) % (2*k) != k:
                    new_A[i] = new_A[i] + [matrix @ A[i] for matrix in word_A[j]]
        for i in range(2*k):
            old_A[i] = remove_duplicates(old_A[i] + word_A[i])
            word_A[i] = remove_duplicates(new_A[i])
        for i in range(2*k):
            word_A[i] = remove_preceding_elements(word_A[i], old_A + word_A[:i])
    for i in range(2*k):
        all_A[i] = old_A[i] + word_A[i]
    together_A = [matrix for matrix_list in all_A for matrix in matrix_list]
    together_A.sort(key=lambda M: np.trace(M.T @ M))
    return together_A
#################################
# Compute the equations for the bisector
def word_bisectors(generators, length, center):
    words = generator_to_words(generators, length)
    wbs = [Word_Bis(word, np.array(word) @ inv(np.array(center)) @ np.array(word).T - inv(np.array(center)))\
           for word in words]
    wbs_filtered = [wb for wb in wbs if not np.all(np.abs(wb.bis)<threshold)]
    return wbs_filtered
################################
# Both are positive definite
def elongate(matrix_1, matrix_2):
    matrix_1 = np.array(matrix_1)
    matrix_2 = np.array(matrix_2)
    if not is_positive_definite(matrix_2):
        raise ValueError("elongate: Input must be positive definite.")
    else:
        matrix = 2*matrix_2 - matrix_1
        while not is_positive_definite(matrix):
            matrix = 0.5*matrix + 0.5*matrix_2
        if not is_positive_definite(matrix):
            print("elongate: unexpected non-positive definite matrix.")
        matrix = matrix/((det(matrix)) ** (1/3))
        return np.array(matrix)
####################################
def first_unique_element(list1, list2):
    set2 = set(list2)  # Convert list2 to a set for fast lookup
    for item in list1:
        if item not in set2:
            return item
    return None  # Return None if no unique element is found
####################################
# Find the perturbation of the matrix.
# Presumably, the matrix is positive definite, and is normal to all old equations and the new equation.
# Find a positive definite matrix, still normal to all old equations, but has a positive product with the new equation.
def perturb_within_plane(matrix, equations, new_equation):
    matrix = np.array(matrix)
    equations = [np.array(equation) for equation in equations]
    new_equation = np.array(new_equation)
    if not is_positive_definite(matrix):
        raise ValueError("perturb_within_plane: Input must be positive definite.")
    else:
        # Compute the projection of the new equation to the old equations.
        # This is with respect to the inner product: matrix^-1*equation.
        matrix_sqrt = sqrtm(matrix)
        equations_trans = [matrix_sqrt @ equ @ matrix_sqrt for equ in equations]
        new_equation_trans = matrix_sqrt @ new_equation @ matrix_sqrt
        orth_equations = orth_matrix(equations_trans)
        orth_vectors = [word_to_vector_new(equation) for equation in orth_equations]
        new_vector = word_to_vector_new(new_equation_trans)
        # Stack the current independent vectors into a matrix
        orth_vectors_matrix = np.array(orth_vectors).T  # Transpose so that each column is a vector
        # Compute the projection of the new vector onto the space spanned by the existing vectors
        coeffs = list(np.linalg.pinv(orth_vectors_matrix) @ new_vector)
        projection_trans = sum(coeff*equ for coeff, equ in zip(coeffs, orth_equations))
        projection = matrix_sqrt @ projection_trans @ matrix_sqrt
        # Take midpoints until the result is
        while not is_positive_definite(projection):
            projection = 0.5*projection + 0.5*matrix
        if not is_positive_definite(projection):
            print("perturb_within_plane: unexpected non-positive definite matrix.")
        projection = projection/((det(projection)) ** (1/3))
        return projection
####################################
def selberg_domain_add_facet(wbs_active, List_faces, new_wb):
    new_vec = word_to_vector(new_wb.bis)
    # Assign to each face a case number
    List_temp = [0]*len(List_faces)
    # After each round, I will always sort the elements so their codimensions are small to large.
    for j in range(len(List_temp)):
        # If the equations defining F_j span the new equation
        face_equs = [wbs_active[ind].bis for ind in List_faces[j].equs]
        face_vecs = [word_to_vector(equ) for equ in face_equs]
        if not is_linearly_independent(linearly_independent_subset(face_vecs), new_vec):
            List_temp[j] = 1
        # If the face is a minimal face. Since excellent me always sorts the faces, I can always check j from small to large.
        elif List_faces[j].subfaces == []:
            # if the new hyperplane intersects with the minimal face, it's type 6
            if find_positive_definite_intersection(face_equs + [new_wb.bis]).is_intersection:
                List_temp[j] = 6
            # if the new hyperplane does not intersect with the minimal face, it's type 2 or 4
            else:
                face_sample_point = List_faces[j].sample_point
                if np.trace(face_sample_point @ new_wb.bis) > 0:
                    List_temp[j] = 2
                else:
                    List_temp[j] = 4
        # If the face has subfaces.
        else:
            face_subfaces = List_faces[j].subfaces
            face_subfaces_temp = [List_temp[ind] for ind in face_subfaces]
            # If the type of either subface is 6.
            if 6 in face_subfaces_temp:
                List_temp[j] = 6
            # If the type of a subface is 2 or 3, while which of the other subface is 4 or 5.
            elif {2, 3} & set(face_subfaces_temp) and {4, 5} & set(face_subfaces_temp):
                List_temp[j] = 6
            # If the type of a subface is 1, 3, or 5.
            elif {1, 3, 5} & set(face_subfaces_temp):
                face_sample_point = List_faces[j].sample_point
                if np.trace(face_sample_point @ new_wb.bis) > 0:
                    List_temp[j] = 3
                else:
                    List_temp[j] = 5
            # Types of all subfaces are 2, or are 4.
            else:
                # If the new hyperplane intersects the span of the face
                if find_positive_definite_intersection(face_equs + [new_wb.bis]).is_intersection:
                    # Sample point of this intersection
                    face_inters_sample_point = find_positive_definite_intersection(face_equs + [new_wb.bis]).sample_point
                    # Find out the equations shape the sides of the face 
                    face_subfaces_equs = []
                    for ind in face_subfaces:
                        if List_faces[ind].codim == List_faces[j].codim + 1:
                            face_subfaces_equs_temp = [elem for elem in List_faces[ind].equs if elem not in List_faces[j].equs]
                            face_subfaces_equs[:] = list(set(face_subfaces_equs) | set(face_subfaces_equs_temp))
                    # Assume the type is 6
                    List_temp[j] = 6
                    for ind in face_subfaces_equs:
                        # However, if any side separates the sample point from the face, the type is either 2 or 4
                        if np.trace(face_inters_sample_point @ wbs_active[ind].bis) < 0:
                            List_temp[j] = face_subfaces_temp[0]
                            break
                # If the new hyperplane does not intersect the span of the face, the type is either 2 or 4
                else:
                    List_temp[j] = face_subfaces_temp[0]
    # If the face is of type 4 or 5, it will be deleted.
    ind_remove_list = [j for j in range(len(List_temp)) if List_temp[j] in [4, 5]]
    for j in sorted(ind_remove_list, reverse=True):
        del List_faces[j]
        del List_temp[j]
    # The presence of these faces in subfaces is also erased.
    for j in range(len(List_temp)):
        List_faces[j].subfaces = [ind for ind in List_faces[j].subfaces if ind not in ind_remove_list]
        List_subfaces_temp = []
        for ind in List_faces[j].subfaces:
            decrease = sum(1 for val in ind_remove_list if val < ind)
            List_subfaces_temp.append(ind - decrease)
        List_faces[j].subfaces = List_subfaces_temp.copy()
    # Update the remaining elements
    for j in range(len(List_temp)):
        # If the face is of type 1, the new equation will be added.
        if List_temp[j] == 1:
            List_faces[j].equs.append(len(wbs_active))
        # If the face is of type 6:
        elif List_temp[j] == 6:
            # Equations for new face
            new_face_equs = List_faces[j].equs + [len(wbs_active)]
            # Codimension of new face
            new_face_codim = List_faces[j].codim + 1
            # Subfaces for both old and new faces
            new_face_subfaces = [ind for ind in List_faces[j].subfaces if List_temp[ind] == 1]
            for ind in List_faces[j].subfaces:
                if ind < len(List_temp):
                    if List_temp[ind] == 6:
                        List_faces[j].subfaces.append(List_faces[ind].subfaces[-1])
                        new_face_subfaces.append(List_faces[ind].subfaces[-1])
            List_faces[j].subfaces.append(len(List_faces))
            # Sample point for the new face
            if len(new_face_subfaces) == 0:
                face_equs = [wbs_active[ind].bis for ind in List_faces[j].equs]
                new_face_sample_point = find_positive_definite_intersection(face_equs + [new_wb.bis]).sample_point
            elif len(new_face_subfaces) >= 2:
                new_face_sample_point = sum((List_faces[ind].sample_point for ind in new_face_subfaces), np.zeros((3, 3)))
                if not is_positive_definite(new_face_sample_point):
                    print("new_face_sample_point: unexpected non-positive definite matrix.")
                new_face_sample_point = new_face_sample_point/((det(new_face_sample_point)) ** (1/3))
            # If only one subface
            else:
                face_equs = [wbs_active[ind].bis for ind in List_faces[j].equs]
                subface_equ_ind = first_unique_element(List_faces[new_face_subfaces[0]].equs, new_face_equs)
                subface_equ = wbs_active[subface_equ_ind].bis
                subface_sample_point = List_faces[new_face_subfaces[0]].sample_point
                new_face_sample_point = perturb_within_plane(subface_sample_point, face_equs + [new_wb.bis], subface_equ)
            # Sample point for the old face
            if np.trace(List_faces[j].sample_point @ new_wb.bis) < np.sqrt(threshold):
                face_equs = [wbs_active[ind].bis for ind in List_faces[j].equs]
                old_face_sample_point = perturb_within_plane(new_face_sample_point, face_equs, new_wb.bis)
                # Check if this point is inside the polytope. 
                # The new face will be good. Moreover, if the old sample point is on the face, it will be fine.
                # To make it safe, add that ind is not in List_faces[j].equs
                while any(ind not in List_faces[j].equs and np.trace(wb.bis @ List_faces[j].sample_point) > threshold\
                          and np.trace(wb.bis @ old_face_sample_point) < threshold for ind, wb in enumerate(wbs_active)):
                    old_face_sample_point = 0.5*(old_face_sample_point + new_face_sample_point)
                temporary_sample_point = 0.5*(old_face_sample_point + new_face_sample_point)
                if not is_positive_definite(temporary_sample_point):
                    print("temporary_sample_point: unexpected non-positive definite matrix.")
                temporary_sample_point = temporary_sample_point/((det(temporary_sample_point)) ** (1/3))
                List_faces[j].sample_point = temporary_sample_point
            # Save the new face to List_faces
            List_faces.append(Poly_Face(new_face_equs, new_face_codim, new_face_subfaces, np.array(new_face_sample_point)))
    # Save the new equation to bises_active
    wbs_active.append(new_wb)
    # Remove the unnecessary equations
    equ_remove_list = list(range(len(wbs_active)))
    for j in range(len(List_faces)):
        if List_faces[j].codim == 1:
            equ_remove_list = [ind for ind in equ_remove_list if ind != List_faces[j].equs[0]]
    for j in sorted(equ_remove_list, reverse=True):
        del wbs_active[j]
    for j in range(len(List_faces)):
        List_faces[j].equs = [ind for ind in List_faces[j].equs if ind not in equ_remove_list]
        List_equs_temp = []
        for ind in List_faces[j].equs:
            decrease = sum(1 for val in equ_remove_list if val < ind)
            List_equs_temp.append(ind - decrease)
        List_faces[j].equs = List_equs_temp.copy()
    # Sort the faces again, including the subfaces
    List_faces_indexed = [(i, face) for i, face in enumerate(List_faces)]
    List_faces_indexed.sort(key=lambda obj: obj[1].codim, reverse=True)
    index_mapping = {old_index: new_index for new_index, (old_index, _) in enumerate(List_faces_indexed)}
    for _, face in List_faces_indexed:
        face.subfaces = [index_mapping[ind] for ind in face.subfaces]
    List_faces = [face for _, face in List_faces_indexed]
    return wbs_active, List_faces
def random_SL3_qr():
    # 1. random Gaussian, QR
    A = np.random.randn(3,3)
    Q, R = np.linalg.qr(A)
    # ensure Q has det +1 (not a reflection)
    if np.linalg.det(Q) < 0:
        Q[:,0] *= -1

    # 2. random diag with zero trace in exponent
    x = np.random.randn(3)
    x -= np.mean(x)           # now x1+x2+x3 = 0
    D = np.diag(np.exp(x))
    # 3. assemble
    return Q @ D
# Check if the word takes the old face to the new one
def equal_spaces(old_equations, new_equations, word):
    # tr(Q^TXQQ^{-1}AQ^{-1}T) = 0
    mapped_equations = [inv(word) @ mat @ inv(word.T) for mat in old_equations]
    mapped_vectors = [word_to_vector(mat) for mat in mapped_equations]
    new_vectors = [word_to_vector(mat) for mat in new_equations]
    rank_A = len(linearly_independent_subset(mapped_vectors))
    rank_B = len(linearly_independent_subset(new_vectors))
    rank_AplusB = len(linearly_independent_subset(mapped_vectors + new_vectors))
    spans_equal = (rank_A == rank_AplusB) and (rank_B == rank_AplusB)
    return spans_equal
def face_is_paired(my_wbs, my_face_list, old_face_ind, new_face_ind, word):
    if my_face_list[old_face_ind].codim == my_face_list[new_face_ind].codim:
        cod = my_face_list[old_face_ind].codim
        if len(my_face_list[old_face_ind].subfaces) == len(my_face_list[new_face_ind].subfaces):
            old_face_equations = [my_wbs[ind].bis for ind in my_face_list[old_face_ind].equs]
            new_face_equations = [my_wbs[ind].bis for ind in my_face_list[new_face_ind].equs]
            if equal_spaces(old_face_equations, new_face_equations, word):
                if len(my_face_list[old_face_ind].subfaces) == 0:
                    return True
                else:
                    old_facets = [j for j in my_face_list[old_face_ind].subfaces if my_face_list[j].codim == cod + 1]
                    new_facets = [k for k in my_face_list[new_face_ind].subfaces if my_face_list[k].codim == cod + 1]
                    # all old facets find a new partner
                    if len(old_facets) != len(new_facets):
                        return False
                    for j in old_facets:
                        if not any(face_is_paired(my_wbs, my_face_list, j, k, word) for k in new_facets):
                            return False
                    return True
            else:
                return False
        else:
            return False
    else:
        return False
def polytope_is_exact(my_wbs, my_face_list):
    # Get a list of facets
    facet_indices = [i for i in range(len(my_face_list)) if my_face_list[i].codim == 1]
    paired_indices = []
    for i in facet_indices:
        i_pair = next((j for j in facet_indices if face_is_paired(my_wbs, my_face_list,\
                                                                  i, j, my_wbs[my_face_list[i].equs[0]].word)), None)
        if i_pair == None:
            return False, facet_indices, None
        else:
            paired_indices.append(i_pair)
    return True, facet_indices, paired_indices
def quick_exact(my_wbs, my_face_list):
    facet_indices = [i for i in range(len(my_face_list)) if my_face_list[i].codim == 1]
    unpaired_ridges = []
    for i in facet_indices:
        i_ridges = [j for j in my_face_list[i].subfaces if my_face_list[j].codim == 2]
        i_pair = next((i_0 for i_0 in facet_indices if np.all(np.abs(my_wbs[my_face_list[i].equs[0]].word @ my_wbs[my_face_list[i_0].equs[0]].word\
                                                                     - np.eye(3))<threshold)), None)
        if i_pair != None:
            i_pair_ridges = [j_0 for j_0 in my_face_list[i_pair].subfaces if my_face_list[j_0].codim == 2]
            for j in i_ridges:
                j_equations = [my_wbs[ind].bis for ind in my_face_list[j].equs]
                j_pair = next((j_0 for j_0 in i_pair_ridges if equal_spaces(j_equations,\
                                                                            [my_wbs[ind].bis for ind in my_face_list[j_0].equs],\
                                                                            my_wbs[my_face_list[i].equs[0]].word)), None)
                if j_pair == None:
                    mat = (my_wbs[my_face_list[i].equs[0]].word).T @ my_face_list[j].sample_point @ my_wbs[my_face_list[i].equs[0]].word
                    if all(np.trace(mat @ my_wbs[ind].bis)> threshold for ind in range(len(my_wbs)) if ind not in my_face_list[i_pair].equs):
                        unpaired_ridges.append([i, j])
    if len(unpaired_ridges) == 0:
        return True, None
    else:
        return False, unpaired_ridges
def compute_ridge_cycle(my_wbs, my_face_list):
    if not polytope_is_exact(my_wbs, my_face_list):
        return None
    else:
        facet_indices = [i for i in range(len(my_face_list)) if my_face_list[i].codim == 1]
        all_ridge_indices = [i for i in range(len(my_face_list)) if my_face_list[i].codim == 2]
        ridge_cycle_list = []
        for i in facet_indices:
            ridge_indices = [j for j in my_face_list[i].subfaces if j in all_ridge_indices]
            for j in ridge_indices:
                if any(j in ridge_cycle.ridge for ridge_cycle in ridge_cycle_list):
                    continue
                else:
                    current_ridge = j
                    current_facet = i
                    current_pairing = my_face_list[current_facet].equs[0]
                    ridge_cycle = Ridge_Cycles([current_ridge], [current_pairing])
                    # Ridge cycles will not be too long
                    for _ in range(2*len(all_ridge_indices)):
                        mapped_facet = next((mapped_i for mapped_i in facet_indices if \
                                          face_is_paired(my_wbs, my_face_list, current_facet, mapped_i, my_wbs[current_pairing].word)), None)
                        new_ridge_indices = [new_j for new_j in my_face_list[mapped_facet].subfaces if new_j in all_ridge_indices]
                        new_ridge = next((new_j for new_j in new_ridge_indices if \
                                          face_is_paired(my_wbs, my_face_list, current_ridge, new_j, my_wbs[current_pairing].word)), None)
                        new_facet = next(new_i for new_i in facet_indices if new_ridge in my_face_list[new_i].subfaces and new_i != mapped_facet)
                        new_pairing = my_face_list[new_facet].equs[0]
                        if new_ridge == ridge_cycle.ridge[0] and new_pairing == ridge_cycle.pairing[0]:
                            ridge_cycle_list.append(ridge_cycle)
                            break
                        else:
                            ridge_cycle.ridge.append(new_ridge)
                            ridge_cycle.pairing.append(new_pairing)
                            current_ridge = new_ridge
                            current_facet = new_facet
                            current_pairing = new_pairing
        return ridge_cycle_list
def Riemannian_angle(equ_1, equ_2, mat):
    comp_1 = mat @ equ_1
    comp_2 = mat @ equ_2
    angle_cos = - (np.trace(comp_1 @ comp_2))/(np.sqrt((np.trace(comp_1 @ comp_1)) * (np.trace(comp_2 @ comp_2))))
    angle = np.arccos(angle_cos)
    return angle
def angle_sum(my_wbs, my_face_list, ridge_cycle):
    angle_sum = 0
    point = my_face_list[ridge_cycle.ridge[0]].sample_point
    for i in range(len(ridge_cycle.ridge)):
        first_bis = my_face_list[ridge_cycle.ridge[i]].equs[0]
        second_bis = my_face_list[ridge_cycle.ridge[i]].equs[1]
        angle = Riemannian_angle(my_wbs[first_bis].bis, my_wbs[second_bis].bis, point)
        angle_sum = angle_sum + angle
        word = my_wbs[ridge_cycle.pairing[i]].word
        point = word.T @ point @ word
    quotient = 2*np.pi/angle_sum
    quotient_round = round(quotient)
    if abs(quotient - quotient_round)> 100*threshold:
        return None
    else:
        return quotient_round
######################################
def compute_selberg_domain(generators, length, center):
    wbs = word_bisectors(generators, length, center)
    # Faces that are utilized in forming the polytope
    wbs_active = []
    # Data for the faces. Initially, we have the entire space as a face.
    List_faces = [Poly_Face([], 0, [], np.array(center))]
    print("number of words:", len(wbs))
    for i in range(len(wbs)):
        new_wb = wbs[i]
        wbs_active, List_faces = selberg_domain_add_facet(wbs_active, List_faces, new_wb)
        print(i, "-th loop completed")
    return wbs_active, List_faces
def compute_selberg_domain_new(generators, length_1, length_2, loop_times, center):
    # Likely need most of these
    wbs = word_bisectors(generators, length_1, center)
    # Only need a few of them. Use refined algorithm to catch these
    more_wbs = word_bisectors(generators, length_2, center)
    # Faces that are utilized in forming the polytope
    wbs_active = []
    # Data for the faces. Initially, we have the entire space as a face.
    List_faces = [Poly_Face([], 0, [], np.array(center))]
    print("number of words:", len(wbs))
    for i in range(len(wbs)):
        new_wb = wbs[i]
        wbs_active, List_faces = selberg_domain_add_facet(wbs_active, List_faces, new_wb)
        print(i, "-th loop completed")
    for _ in range(loop_times):
        is_exact, unpaired_ridges = quick_exact(wbs_active, List_faces)
        if is_exact:
            break
        else:
            facet_indices = [i for i in range(len(List_faces)) if List_faces[i].codim == 1]
            print("current number of facets:", len(facet_indices))
            print("current number of unpaired ridges", len(unpaired_ridges))
            # find my facet with least trace
            min_trace = np.inf
            new_wb = None
            for i, j in unpaired_ridges:
                i_pair = next((i_0 for i_0 in facet_indices if np.all(np.abs(wbs_active[List_faces[i].equs[0]].word @ wbs_active[List_faces[i_0].equs[0]].word\
                                                                     - np.eye(3))<threshold)), None)
                j_equations = [wbs_active[ind].bis for ind in List_faces[j].equs]
                i_pair_equation = wbs_active[List_faces[i_pair].equs[0]].bis
                for k in range(len(more_wbs)):
                    if np.trace(more_wbs[k].bis @ np.array(center)) < min_trace:
                        candidate_equations = [i_pair_equation, more_wbs[k].bis]
                        if equal_spaces(j_equations, candidate_equations, wbs_active[List_faces[i].equs[0]].word):
                            min_trace = np.trace(more_wbs[k].bis @ np.array(center))
                            new_wb = more_wbs[k]
            if new_wb is not None:
                wbs_active, List_faces = selberg_domain_add_facet(wbs_active, List_faces, new_wb)
            print("ridge fixed")
    return wbs_active, List_faces

CPU times: user 3.99 s, sys: 817 ms, total: 4.81 s
Wall time: 4.89 s


In [2]:
#####################################
# generators = [[[0.5, 0.5, 0],
#          [0.5, -0.5, 1],
#          [0.5, -0.5, -1]],
#               [[-0.5, 1, 0.5],
#          [-0.5, -1, 0.5],
#          [0.5, 0, 0.5]],
#              [[-1, 0.5, -0.5],
#          [0, 0.5, 0.5],
#          [1, 0.5, -0.5]]]
# # Test: Compute the polytope structure
generators = [[[0.5, 0.5, 0],
         [0.5, -0.5, 1],
         [0.5, -0.5, -1]],
              [[-0.5, 1, 0.5],
         [-0.5, -1, 0.5],
         [0.5, 0, 0.5]]]
#################################
# Test: Compute the polytope structure
# generators = [[[1, 2, 0],
#                [0, 1, 0],
#                [0, 0, 1]],
#               [[1, 0, 2],
#                [0, 1, 0],
#                [0, 0, 1]],
#               [[1, 0, 0],
#                [0, 1, 2],
#                [0, 0, 1]],
#               [[1, 0, 0],
#                [2, 1, 0],
#                [0, 0, 1]],
#               [[1, 0, 0],
#                [0, 1, 0],
#                [2, 0, 1]],
#               [[1, 0, 0],
#                [0, 1, 0],
#                [0, 2, 1]]]
center = [[1, 0, 0],
         [0, 1, 0],
         [0, 0, 1]]
my_wbs, my_face_list = compute_selberg_domain(generators, 2, center)
# my_wbs, my_face_list = compute_selberg_domain_new(generators, 1, 4, 20, center)
# ############################# 
# Count the number of faces
# All edges and four faces of codimension 3 are on the Satake boundary, not computed here
print("Number of facets:", sum(1 for face in my_face_list if face.codim == 1))
print("Number of ridges:", sum(1 for face in my_face_list if face.codim == 2))
print("Number of faces of codimension 3:", sum(1 for face in my_face_list if face.codim == 3))
############################# 
# Make sure every sample point lies on the corresponding plane. Expected to be something close to zero.
max_trace = 0
for face in my_face_list:
    for ind in face.equs:
        my_trace = np.trace(my_wbs[ind].bis @ face.sample_point)
        max_trace = max(max_trace, abs(my_trace))
print("The largest trace for sample points multiplying with corresponding facets:", max_trace)
############################# 
# Make sure every sample point is in the interior. Expected to be a positive number.
min_trace = np.inf
for face in my_face_list:
    for ind in range(len(my_wbs)):
        if ind not in face.equs:
            my_trace = np.trace(my_wbs[ind].bis @ face.sample_point)
            min_trace = min(min_trace, my_trace)
print("The smallest trace of sample points multiplying with non-corresponding facets:", min_trace)
# print(my_words)

number of words: 16
0 -th loop completed
1 -th loop completed
2 -th loop completed
3 -th loop completed
4 -th loop completed
5 -th loop completed
6 -th loop completed
7 -th loop completed
8 -th loop completed
9 -th loop completed
10 -th loop completed
11 -th loop completed
12 -th loop completed
13 -th loop completed
14 -th loop completed
15 -th loop completed
Number of facets: 6
Number of ridges: 15
Number of faces of codimension 3: 16
The largest trace for sample points multiplying with corresponding facets: 2.886579864025407e-15
The smallest trace of sample points multiplying with non-corresponding facets: 0.08695819189645349


In [3]:
is_exact, original_facets, paired_facets = polytope_is_exact(my_wbs, my_face_list)
print(is_exact)
if is_exact:
    ridge_cycles = compute_ridge_cycle(my_wbs, my_face_list)
    print(ridge_cycles)
    for ridge_cycle in ridge_cycles:
        print(angle_sum(my_wbs, my_face_list, ridge_cycle))

True
[Ridge_Cycles(ridge=[16, 19, 29], pairing=[0, 0, 5]), Ridge_Cycles(ridge=[17, 20, 30], pairing=[0, 1, 4]), Ridge_Cycles(ridge=[22, 21, 24], pairing=[0, 2, 2]), Ridge_Cycles(ridge=[26, 25, 18], pairing=[0, 4, 1]), Ridge_Cycles(ridge=[23, 27, 28], pairing=[1, 1, 2])]
2
1
2
2
2


In [5]:
codim_3_list = [my_face for my_face in my_face_list if my_face.codim == 3]

In [6]:
codim_3 = codim_3_list[0]
for equ in codim_3.equs:
    print(my_wbs[equ].bis)

[[-0.5  0.   0. ]
 [ 0.   0.5 -0.5]
 [ 0.  -0.5  0.5]]
[[ 0.5 -0.5  0. ]
 [-0.5  0.5  0. ]
 [ 0.   0.  -0.5]]
[[-0.5  0.   0. ]
 [ 0.   0.5  0.5]
 [ 0.   0.5  0.5]]


In [8]:
ridge_cycle = ridge_cycles[1]
for wb in ridge_cycle.pairing:
    print(my_wbs[wb].word)

[[ 0.5  0.5  0. ]
 [ 0.5 -0.5  1. ]
 [ 0.5 -0.5 -1. ]]
[[-0.5  1.   0.5]
 [-0.5 -1.   0.5]
 [ 0.5  0.   0.5]]
[[-1.   0.5 -0.5]
 [ 0.   0.5  0.5]
 [ 1.   0.5 -0.5]]


In [12]:
vertices = [[[1, 0, 0],
                    [0, 0, 0],
                    [0, 0, 0]],
                   [[0, 0, 0],
                    [0, 1, 0],
                    [0, 0, 0]],
                   [[0, 0, 0],
                    [0, 0, 0],
                    [0, 0, 1]],
                   [[1, 1, 1],
                    [1, 1, 1],
                    [1, 1, 1]],
                   [[1, -1, -1],
                    [-1, 1, 1],
                    [-1, 1, 1]],
                   [[1, -1, 1],
                    [-1, 1, -1],
                    [1, -1, 1]],
                   [[1, 1, -1],
                    [1, 1, -1],
                    [-1, -1, 1]]]
for inds in itertools.combinations(range(7),5):
    points = [vertices[i] for i in inds]
    plane_equations = orth_matrix(points)
    if len(plane_equations) == 1:
        plane_equation = plane_equations[0]
        if all(np.trace(plane_equation)*np.trace(plane_equation @ mat) > -1e-10 for mat in vertices):
            print(inds)
            print(plane_equation)
            print(det(plane_equation))

(0, 1, 2, 3, 4)
[[ 0.          0.35355339 -0.35355339]
 [ 0.35355339  0.          0.        ]
 [-0.35355339  0.          0.        ]]
0.0
(0, 1, 2, 3, 5)
[[ 0.         -0.35355339  0.        ]
 [-0.35355339  0.          0.35355339]
 [ 0.          0.35355339  0.        ]]
0.0
(0, 1, 2, 3, 6)
[[ 0.          0.         -0.35355339]
 [ 0.          0.          0.35355339]
 [-0.35355339  0.35355339  0.        ]]
0.0
(0, 1, 2, 4, 5)
[[0.         0.         0.35355339]
 [0.         0.         0.35355339]
 [0.35355339 0.35355339 0.        ]]
0.0
(0, 1, 2, 4, 6)
[[0.         0.35355339 0.        ]
 [0.35355339 0.         0.35355339]
 [0.         0.35355339 0.        ]]
0.0
(0, 1, 2, 5, 6)
[[ 0.         -0.35355339 -0.35355339]
 [-0.35355339  0.          0.        ]
 [-0.35355339  0.          0.        ]]
0.0
(0, 1, 3, 4, 5)
[[ 0.00000000e+00 -2.50000000e-01  2.50000000e-01]
 [-2.50000000e-01 -2.98518123e-16  2.50000000e-01]
 [ 2.50000000e-01  2.50000000e-01 -5.00000000e-01]]
-1.908195823574485e-

In [ ]:
######################
# Test: Find the sample point in bad case
# Mat_A = [[1, 0, 0],
#          [0, -1, 0],
#          [0, 0, 0]]
# Mat_B = [[[0, 0, 0],
#           [0, -1, 0],
#           [0, 0, 1]],
#          [[0, 1, 0],
#           [1, -1, 0],
#           [0, 0, 1]],
#          [[0, 0, 0],
#           [0, 0, 1],
#           [0, 1, 0]],
#          [[0, 1, 1],
#           [1, 0, 1],
#           [1, 1, 0]],
#          [[0, 0, 1],
#           [0, 0, 1],
#           [1, 1, 0]],
#          [[0, 1, 0],
#           [1, 0, 0],
#           [0, 0, 0]]]
# Q = np.array(random_SL3_qr())
# M_1 = Q.T @ Mat_A @ Q
# M_2 = Q.T @ Mat_B[0] @ Q
M_1 = [[ 1., -1., -1.],
       [-1.,  0.,  1.],
       [-1.,  1.,  1.]]
M_2 = [[0., -1., -1.],
       [-1.,  1.,  1.],
       [-1.,  1.,  1.]]
M_3 = [[0.,  1., -1.],
       [ 1.,  1., -1.],
       [-1., -1.,  1.]]
words = [M_1, M_2, M_3]
%time X = find_positive_definite_intersection(words).sample_point
print(X)
# Y = Q @ X @ Q.T
# print(Y)

In [10]:
for i, j in itertools.combinations(range(5),2):
    print(i," ",j)

0   1
0   2
0   3
0   4
1   2
1   3
1   4
2   3
2   4
3   4
